In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')
import string
import re
%matplotlib inline
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('stopwords')
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is

In [2]:
health = pd.read_csv('health_tweets.csv')
# health = health.dropna()
health.head()

,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0,Med-Tech World,Malta,Med-Tech World highlights leading innovations ...,245,1090184482047619073,2022-11-18 16:26:39+00:00,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,[None],HubSpot
1,1,Martin Taylor,United Kingdom,Deputy CEO and Co-Founder of the Redwood Techn...,52,4096820361,2022-11-18 16:26:05+00:00,"Congratulations to West Yorkshire ICS, who won...",[['HSJAwards']],Oktopost
2,2,Jits Langedijk,"Utrecht, Nederland",Sr. Solutions Architect @NVIDIA https://t.co/d...,2816,115660165,2022-11-18 16:25:51+00:00,Calling #healthcare #AI professionals! NVIDIA ...,"[['healthcare', 'AI', 'UMC']]",Dynamic Signal
3,3,HealthCare CE,NaN,"Health Care News, Topics, Updates & Reviews of...",0,1566589962035666944,2022-11-18 16:25:18+00:00,"Your immune system may be weakened over time, ...",[None],TweetDeck
4,4,CCS,"Dallas, TX",CCS is a leading provider of collaborative car...,1195,58493082,2022-11-18 16:25:11+00:00,#CCS is growing! We are #hiring for a Customer...,"[['CCS', 'hiring']]",Hootsuite Inc.


In [3]:
len(health)

10000

Roberta Analysis

In [4]:
rob_health = list()
# new =pd.DataFrame()
for i,  row in tqdm(health.iterrows(), total = len(health)):
    try:
#         print({i})
        text = row['text']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_health.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
rob_df2 = pd.DataFrame(rob_health)
rob_df2

,roberta_neg,roberta_neu,roberta_pos
0,0.013511,0.717215,0.269274
1,0.000800,0.018786,0.980414
2,0.001964,0.235552,0.762485
3,0.051754,0.578570,0.369677
4,0.001423,0.144184,0.854392
...,...,...,...
9995,0.000763,0.016908,0.982328
9996,0.038972,0.782525,0.178503
9997,0.002017,0.174261,0.823722
9998,0.004035,0.359459,0.636506


In [6]:
rob_health2 = rob_df2.join(health)

In [7]:
rob_health2

,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0.013511,0.717215,0.269274,0,Med-Tech World,Malta,Med-Tech World highlights leading innovations ...,245,1090184482047619073,2022-11-18 16:26:39+00:00,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,[None],HubSpot
1,0.000800,0.018786,0.980414,1,Martin Taylor,United Kingdom,Deputy CEO and Co-Founder of the Redwood Techn...,52,4096820361,2022-11-18 16:26:05+00:00,"Congratulations to West Yorkshire ICS, who won...",[['HSJAwards']],Oktopost
2,0.001964,0.235552,0.762485,2,Jits Langedijk,"Utrecht, Nederland",Sr. Solutions Architect @NVIDIA https://t.co/d...,2816,115660165,2022-11-18 16:25:51+00:00,Calling #healthcare #AI professionals! NVIDIA ...,"[['healthcare', 'AI', 'UMC']]",Dynamic Signal
3,0.051754,0.578570,0.369677,3,HealthCare CE,NaN,"Health Care News, Topics, Updates & Reviews of...",0,1566589962035666944,2022-11-18 16:25:18+00:00,"Your immune system may be weakened over time, ...",[None],TweetDeck
4,0.001423,0.144184,0.854392,4,CCS,"Dallas, TX",CCS is a leading provider of collaborative car...,1195,58493082,2022-11-18 16:25:11+00:00,#CCS is growing! We are #hiring for a Customer...,"[['CCS', 'hiring']]",Hootsuite Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000763,0.016908,0.982328,9995,World Cord Blood Day,Worldwide,15 NOVEMBER is World Cord Blood Day 2022. Free...,3710,774570569576701952,2022-11-15 13:20:48+00:00,Making #Arizona #Proud! Today is officially #C...,"[['Arizona', 'Proud', 'CordBlood', 'AZ', 'AZGo...",Buffer
9996,0.038972,0.782525,0.178503,9996,Mark Oborn,West Dorset,💡Digital Marketing Consultant \n👊Life and Busi...,1364,15093953,2022-11-15 13:20:23+00:00,The latest Daily Dose of Dentistry! https://t....,"[['healthcare', 'news']]",Paper.li
9997,0.002017,0.174261,0.823722,9997,LLRX,NaN,"LLRX - free indie e-journal on law, tech, KM, ...",7049,16901001,2022-11-15 13:19:01+00:00,"Soft robotic device stimulates muscles, sparks...","[['ALS', 'MS']]",Twitter Web App
9998,0.004035,0.359459,0.636506,9998,CX Scoop,NaN,Get the latest scoop on B2B news and trends in...,4,1576882290222825472,2022-11-15 13:18:34+00:00,.@twilio has revealed its Twilio Segment for H...,[None],Twitter Web App


VADERS Analyis

In [8]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()

In [9]:
# res = {}
v_health = list()
# new =pd.DataFrame()
for i,  row in tqdm(health.iterrows(), total = len(health)):
    text = row['text']
#     myid = row['reviewerID']
    res = sia.polarity_scores(text)
#     print(res)
    v_health.append(res)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
len(v_health)

10000

In [11]:
vaders_health = pd.DataFrame(v_health)
vaders_health

,neg,neu,pos,compound
0,0.000,1.000,0.000,0.0000
1,0.000,0.651,0.349,0.9565
2,0.000,0.815,0.185,0.6800
3,0.044,0.648,0.309,0.8979
4,0.000,0.878,0.122,0.5972
...,...,...,...,...
9995,0.000,0.869,0.131,0.6792
9996,0.000,0.747,0.253,0.5255
9997,0.000,0.754,0.246,0.5994
9998,0.000,1.000,0.000,0.0000


Combine both datasets

In [12]:
final_df2 = vaders_health.join(rob_health2)

In [13]:
final_df2

,neg,neu,pos,compound,roberta_neg,roberta_neu,roberta_pos,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source
0,0.000,1.000,0.000,0.0000,0.013511,0.717215,0.269274,0,Med-Tech World,Malta,Med-Tech World highlights leading innovations ...,245,1090184482047619073,2022-11-18 16:26:39+00:00,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,[None],HubSpot
1,0.000,0.651,0.349,0.9565,0.000800,0.018786,0.980414,1,Martin Taylor,United Kingdom,Deputy CEO and Co-Founder of the Redwood Techn...,52,4096820361,2022-11-18 16:26:05+00:00,"Congratulations to West Yorkshire ICS, who won...",[['HSJAwards']],Oktopost
2,0.000,0.815,0.185,0.6800,0.001964,0.235552,0.762485,2,Jits Langedijk,"Utrecht, Nederland",Sr. Solutions Architect @NVIDIA https://t.co/d...,2816,115660165,2022-11-18 16:25:51+00:00,Calling #healthcare #AI professionals! NVIDIA ...,"[['healthcare', 'AI', 'UMC']]",Dynamic Signal
3,0.044,0.648,0.309,0.8979,0.051754,0.578570,0.369677,3,HealthCare CE,NaN,"Health Care News, Topics, Updates & Reviews of...",0,1566589962035666944,2022-11-18 16:25:18+00:00,"Your immune system may be weakened over time, ...",[None],TweetDeck
4,0.000,0.878,0.122,0.5972,0.001423,0.144184,0.854392,4,CCS,"Dallas, TX",CCS is a leading provider of collaborative car...,1195,58493082,2022-11-18 16:25:11+00:00,#CCS is growing! We are #hiring for a Customer...,"[['CCS', 'hiring']]",Hootsuite Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000,0.869,0.131,0.6792,0.000763,0.016908,0.982328,9995,World Cord Blood Day,Worldwide,15 NOVEMBER is World Cord Blood Day 2022. Free...,3710,774570569576701952,2022-11-15 13:20:48+00:00,Making #Arizona #Proud! Today is officially #C...,"[['Arizona', 'Proud', 'CordBlood', 'AZ', 'AZGo...",Buffer
9996,0.000,0.747,0.253,0.5255,0.038972,0.782525,0.178503,9996,Mark Oborn,West Dorset,💡Digital Marketing Consultant \n👊Life and Busi...,1364,15093953,2022-11-15 13:20:23+00:00,The latest Daily Dose of Dentistry! https://t....,"[['healthcare', 'news']]",Paper.li
9997,0.000,0.754,0.246,0.5994,0.002017,0.174261,0.823722,9997,LLRX,NaN,"LLRX - free indie e-journal on law, tech, KM, ...",7049,16901001,2022-11-15 13:19:01+00:00,"Soft robotic device stimulates muscles, sparks...","[['ALS', 'MS']]",Twitter Web App
9998,0.000,1.000,0.000,0.0000,0.004035,0.359459,0.636506,9998,CX Scoop,NaN,Get the latest scoop on B2B news and trends in...,4,1576882290222825472,2022-11-15 13:18:34+00:00,.@twilio has revealed its Twilio Segment for H...,[None],Twitter Web App


In [14]:
final_df2.to_csv('healthcare_tweets_analysis.csv')

Cleaned

In [2]:
health = pd.read_csv('health_tweets.csv')

In [4]:
# Remove Punctuations
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

health['text2'] = health['text'].apply(lambda x: remove_punct(x))

health

,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source,text2
0,0,Med-Tech World,Malta,Med-Tech World highlights leading innovations ...,245,1090184482047619073,2022-11-18 16:26:39+00:00,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,[None],HubSpot,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...
1,1,Martin Taylor,United Kingdom,Deputy CEO and Co-Founder of the Redwood Techn...,52,4096820361,2022-11-18 16:26:05+00:00,"Congratulations to West Yorkshire ICS, who won...",[['HSJAwards']],Oktopost,Congratulations to West Yorkshire ICS who won ...
2,2,Jits Langedijk,"Utrecht, Nederland",Sr. Solutions Architect @NVIDIA https://t.co/d...,2816,115660165,2022-11-18 16:25:51+00:00,Calling #healthcare #AI professionals! NVIDIA ...,"[['healthcare', 'AI', 'UMC']]",Dynamic Signal,Calling healthcare AI professionals NVIDIA and...
3,3,HealthCare CE,NaN,"Health Care News, Topics, Updates & Reviews of...",0,1566589962035666944,2022-11-18 16:25:18+00:00,"Your immune system may be weakened over time, ...",[None],TweetDeck,Your immune system may be weakened over time t...
4,4,CCS,"Dallas, TX",CCS is a leading provider of collaborative car...,1195,58493082,2022-11-18 16:25:11+00:00,#CCS is growing! We are #hiring for a Customer...,"[['CCS', 'hiring']]",Hootsuite Inc.,CCS is growing We are hiring for a Customer Se...
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,World Cord Blood Day,Worldwide,15 NOVEMBER is World Cord Blood Day 2022. Free...,3710,774570569576701952,2022-11-15 13:20:48+00:00,Making #Arizona #Proud! Today is officially #C...,"[['Arizona', 'Proud', 'CordBlood', 'AZ', 'AZGo...",Buffer,Making Arizona Proud Today is officially CordB...
9996,9996,Mark Oborn,West Dorset,💡Digital Marketing Consultant \n👊Life and Busi...,1364,15093953,2022-11-15 13:20:23+00:00,The latest Daily Dose of Dentistry! https://t....,"[['healthcare', 'news']]",Paper.li,The latest Daily Dose of Dentistry httpstcosiU...
9997,9997,LLRX,NaN,"LLRX - free indie e-journal on law, tech, KM, ...",7049,16901001,2022-11-15 13:19:01+00:00,"Soft robotic device stimulates muscles, sparks...","[['ALS', 'MS']]",Twitter Web App,Soft robotic device stimulates muscles sparks ...
9998,9998,CX Scoop,NaN,Get the latest scoop on B2B news and trends in...,4,1576882290222825472,2022-11-15 13:18:34+00:00,.@twilio has revealed its Twilio Segment for H...,[None],Twitter Web App,twilio has revealed its Twilio Segment for Hea...


In [5]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

health['text3'] = health['text2'].apply(lambda x: tokenization(x.lower()))
health.head()

,Unnamed: 0,user_name,user_location,user_description,Number of Favorites,ID,date,text,hashtags,source,text2,text3
0,0,Med-Tech World,Malta,Med-Tech World highlights leading innovations ...,245,1090184482047619073,2022-11-18 16:26:39+00:00,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,[None],HubSpot,THE HEALTH OF OUR SOCIETY IS INTRINSICALLY LIN...,"[the, health, of, our, society, is, intrinsica..."
1,1,Martin Taylor,United Kingdom,Deputy CEO and Co-Founder of the Redwood Techn...,52,4096820361,2022-11-18 16:26:05+00:00,"Congratulations to West Yorkshire ICS, who won...",[['HSJAwards']],Oktopost,Congratulations to West Yorkshire ICS who won ...,"[congratulations, to, west, yorkshire, ics, wh..."
2,2,Jits Langedijk,"Utrecht, Nederland",Sr. Solutions Architect @NVIDIA https://t.co/d...,2816,115660165,2022-11-18 16:25:51+00:00,Calling #healthcare #AI professionals! NVIDIA ...,"[['healthcare', 'AI', 'UMC']]",Dynamic Signal,Calling healthcare AI professionals NVIDIA and...,"[calling, healthcare, ai, professionals, nvidi..."
3,3,HealthCare CE,NaN,"Health Care News, Topics, Updates & Reviews of...",0,1566589962035666944,2022-11-18 16:25:18+00:00,"Your immune system may be weakened over time, ...",[None],TweetDeck,Your immune system may be weakened over time t...,"[your, immune, system, may, be, weakened, over..."
4,4,CCS,"Dallas, TX",CCS is a leading provider of collaborative car...,1195,58493082,2022-11-18 16:25:11+00:00,#CCS is growing! We are #hiring for a Customer...,"[['CCS', 'hiring']]",Hootsuite Inc.,CCS is growing We are hiring for a Customer Se...,"[ccs, is, growing, we, are, hiring, for, a, cu..."


In [6]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
health['text4'] = health['text3'].apply(lambda x: remove_stopwords(x))


In [8]:
# health.to_csv('healthcare_tweets_cleaned.csv')

In [9]:
health = pd.read_csv('healthcare_tweets_cleaned.csv')

In [10]:
rob_health = list()
# new =pd.DataFrame()
for i,  row in tqdm(health.iterrows(), total = len(health)):
    try:
#         print({i})
        text = row['text4']
        encoded_text= tokenizer(text, return_tensors = 'pt')
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'roberta_neg' : scores[0],
            'roberta_neu' : scores[1],
            'roberta_pos' : scores[2]
        }
        rob_health.append(scores_dict) 
    except RuntimeError:
        print(f'Too Long Text')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
rob_df3 = pd.DataFrame(rob_health)

In [12]:
rob_health2 = rob_df3.join(health)

In [13]:
rob_health2['text5'] = rob_health2['text4'].apply(lambda text4:sia.polarity_scores(str(text4)))

objs = [rob_health2, pd.DataFrame(rob_health2['text5'].tolist()).iloc[:, :4]]
rob_health3 = pd.concat(objs, axis=1).drop('text5', axis=1)

In [14]:
rob_health3.to_csv('healthcare_tweets_analysis_cleaned.csv')